In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import necessary libraries
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI # Or replace with your preferred LLM provider
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
from crewai_tools import BaseTool # For custom tool creation

# Load environment variables from .env file
load_dotenv()
print("Environment variables loaded from .env file.")

In [4]:
# --- Verify API Key and Configure LLM ---

# For OpenAI (uncomment and use if you have an OpenAI key)
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OpenAI API Key loaded successfully!")
    llm_config = LLM(
        model="openai/gpt-4o",  # You can choose other models like "gpt-3.5-turbo"
        api_key=openai_api_key
    )
    print("CrewAI LLM configured with OpenAI.")
else:
    print("OpenAI API Key NOT found. Checking for Google Gemini key...")
    llm_config = None # Initialize to None if no OpenAI key

# For Google Gemini (uncomment and use if you have a Gemini key AND no OpenAI key)
if llm_config is None: # Only try Gemini if OpenAI wasn't configured
    gemini_api_key = os.getenv("GEMINI_API_KEY")
    if gemini_api_key:
        print("Google Gemini API Key loaded successfully!")
        llm_config = LLM(
            model="gemini/gemini-pro", # or "gemini/gemini-1.5-pro"
            api_key=gemini_api_key
        )
        print("CrewAI LLM configured with Google Gemini.")
    else:
        print("Google Gemini API Key NOT found. No LLM configured. Please set one in your .env file.")

if llm_config is None:
    raise ValueError("No valid API key found for OpenAI or Google Gemini. Please set one in your .env file.")

# You will now use 'llm_config' when defining your agents
print("\nReady to define Agents and Tasks!")

OpenAI API Key loaded successfully!
CrewAI LLM configured with OpenAI.

Ready to define Agents and Tasks!


## Define Input Data

In [ ]:
def extract_candidate_name(resume_text: str) -> str:
    """Extracts a candidate's name from the resume text (simplified)."""
    # A simple heuristic: assume the first line is the name
    first_line = resume_text.strip().split('\n')[0]
    return first_line.replace('Anna Chen', 'Anna Chen').strip() # Placeholder for demo consistency


# ### **2. Buyside Asset Management Skill Graph**
# ##This class encapsulates your deep domain knowledge about buyside skills.Job details consists of three parts - Job description, company description, and hiring manager preference.

In [ ]:
class BuysideAssetManagementSkillGraph:
    def __init__(self):
        # The core of our intelligent skill graph, structured by categories and details
        self.graph = {
            "Investment Research & Analysis": {
                "type": "Core Capability",
                "aliases": ["equity research", "credit research", "fixed income analysis", "research"],
                "sub_skills": {
                    "Fundamental Analysis": {
                        "aliases": ["company analysis", "industry analysis", "economic research", "basic research"],
                        "methods": ["SWOT analysis", "Porter's Five Forces", "Competitive Landscape"],
                        "key_inputs": ["financial statements", "management calls", "industry reports"]
                    },
                    "Quantitative Analysis": {
                        "aliases": ["quant research", "statistical analysis", "data-driven insights", "systematic strategies"],
                        "tools": ["Python (Pandas, NumPy, SciPy)", "R", "Matlab", "SQL", "Bloomberg Terminal", "Refinitiv Eikon"],
                        "applications": ["factor modeling", "alpha generation", "risk attribution", "backtesting"]
                    }
                },
                "output": "Investment Thesis",
                "related_roles": ["Research Analyst", "Portfolio Manager (Research Focus)"]
            },
            "Financial Modeling & Valuation": {
                "type": "Technical Skill",
                "aliases": ["valuation modeling", "equity valuation", "debt valuation", "financial models", "modeling"],
                "models": {
                    "DCF Valuation": {
                        "aliases": ["discounted cash flow", "dcf"],
                        "components": ["free cash flow projection", "WACC calculation", "terminal value"]
                    },
                    "Comparable Company Analysis": {
                        "aliases": ["comps", "multiples analysis", "comparable analysis"],
                        "metrics": ["EV/EBITDA", "P/E", "P/B", "Dividend Yield"]
                    },
                    "LBO Analysis": {
                        "aliases": ["leveraged buyout model", "lbo"],
                        "context": "Private Equity, M&A"
                    },
                    "Accretion/Dilution Analysis": {
                        "aliases": ["a&d model"],
                        "context": "M&A"
                    }
                },
                "tools": {"Microsoft Excel": {"level_required": "advanced", "specifics": ["VBA", "Power Query"]}, "Python": {"libraries": ["Numpy", "Scipy"]}},
                "applications": ["Investment Decision Making", "M&A Advisory", "Capital Allocation"],
                "required_skills": ["Accounting Principles", "Corporate Finance Concepts"]
            },
            "Portfolio Management & Construction": {
                "type": "Core Capability",
                "aliases": ["asset allocation", "portfolio optimization", "risk management", "portfolio construction"],
                "sub_skills": {
                    "Risk Management": {
                        "aliases": ["risk assessment", "hedging strategies", "VaR", "risk analysis"],
                        "tools": ["Python (Backtesting, RiskMetrics)", "Matlab"]
                    },
                    "Asset Allocation": {
                        "aliases": ["strategic asset allocation", "tactical asset allocation"],
                        "models": ["Modern Portfolio Theory (MPT)", "Black-Litterman"]
                    },
                    "Performance Attribution": {
                        "aliases": ["performance analysis", "attribution analysis"]
                    }
                },
                "output": "Optimized Portfolio",
                "related_roles": ["Portfolio Manager", "Risk Manager", "Quant Analyst"]
            },
            "Communication & Presentation": {
                "type": "Soft Skill/Interpersonal",
                "aliases": ["client presentation", "stakeholder communication", "verbal & written communication", "presentation skills", "public speaking"],
                "applications": ["presenting investment recommendations", "client pitches", "internal committee briefings", "investor relations"],
                "effectiveness_indicators": ["clarity", "persuasiveness", "ability to handle Q&A", "tailoring message to audience", "storytelling"],
                "related_roles": ["Client Relationship Manager", "Portfolio Manager", "Senior Analyst"]
            },
            "Market & Sector Expertise": {
                "type": "Domain Knowledge",
                "aliases": ["industry expertise", "sector knowledge", "market understanding"],
                "examples": ["TMT (Tech, Media, Telecom)", "Healthcare", "Financials", "Energy", "Consumer Goods", "Real Estate"],
                "depth_indicators": ["understanding of value chains", "regulatory landscape", "key players", "future trends", "competitive dynamics"],
                "related_roles": ["Sector Analyst", "Specialist Portfolio Manager"]
            },
            "Due Diligence": {
                "type": "Process Skill",
                "aliases": ["dd", "investment due diligence"],
                "applications": ["private equity investments", "M&A", "alternative investments"],
                "key_components": ["financial due diligence", "commercial due diligence", "legal due diligence", "operational due diligence"],
                "required_skills": ["Financial Analysis", "Critical Thinking"]
            }
        }

    def _enrich_skill_data(self, skill_id, skill_data, detected_phrase, context=""):
        """Helper to enrich the detected skill with context and specific details."""
        enriched_data = {
            "name": skill_id,
            "detected_phrase": detected_phrase,
            "type": skill_data.get("type", "Unknown"),
            "level": None, # Will be determined by context/keywords
            "applications": skill_data.get("applications", []),
            "related_skills": skill_data.get("required_skills", []) + skill_data.get("related_skills", []), # Combine required and related
            "context_keywords": [], # Keywords from context indicating depth/application
            "details": {} # For specific models, tools, sectors, etc.
        }

        lower_context = context.lower()

        # General level determination based on action verbs in context
        if "developed" in lower_context or "built" in lower_context or "led" in lower_context or "architected" in lower_context or "pioneered" in lower_context:
            enriched_data["level"] = "advanced/lead"
        elif "conducted" in lower_context or "managed" in lower_context or "optimized" in lower_context or "implemented" in lower_context or "performed" in lower_context:
            enriched_data["level"] = "intermediate/applied"
        elif "assisted" in lower_context or "participated" in lower_context or "familiar with" in lower_context or "used" in lower_context or "gained experience in" in lower_context:
            enriched_data["level"] = "basic/familiar"
        
        # If no specific action verb suggests level, default based on type or assume familiar
        if enriched_data["level"] is None:
            if enriched_data["type"] == "Core Capability" or enriched_data["type"] == "Technical Skill":
                enriched_data["level"] = "intermediate" # Assume average proficiency if no strong indicators
            elif enriched_data["type"] == "Domain Knowledge" or enriched_data["type"] == "Soft Skill/Interpersonal":
                enriched_data["level"] = "familiar" # Assume familiarity for knowledge/soft skills without strong evidence

        # Specific enrichments based on skill_id
        if skill_id == "Financial Modeling & Valuation":
            for model_name, model_data in skill_data.get("models", {}).items():
                if model_name.lower() in lower_context or any(a.lower() in lower_context for a in model_data.get("aliases", [])):
                    enriched_data["details"]["Valuation_Model"] = model_name
                    if "components" in model_data:
                        enriched_data["details"]["Model_Components_Mentioned"] = [c for c in model_data["components"] if c.lower() in lower_context]
                    if "context" in model_data and model_data["context"].lower() in lower_context:
                        enriched_data["details"]["Model_Context"] = model_data["context"]
                    break # Found the most specific model, stop
            if "excel" in lower_context:
                enriched_data["details"]["Tool"] = "Microsoft Excel"
            if "python" in lower_context:
                enriched_data["details"]["Tool"] = "Python"
        
        if skill_id == "Investment Research & Analysis":
            # Better way to detect sector if it's not a primary skill_id
            # This logic might need to rely more on 'Market & Sector Expertise' being detected separately
            # For simplicity in this demo, we'll check common sector keywords in context
            detected_sector = None
            for sector_key in self.graph.get("Market & Sector Expertise", {}).get("examples", []):
                if sector_key.lower().split(" ")[0] in lower_context:
                    detected_sector = sector_key
                    break
            if detected_sector:
                enriched_data["details"]["Sector"] = detected_sector

            for sub_skill, sub_data in skill_data.get("sub_skills", {}).items():
                if sub_skill.lower() in lower_context or any(a.lower() in lower_context for a in sub_data.get("aliases", [])):
                    enriched_data["details"]["Research_Type"] = sub_skill
                    break
            
        if skill_id == "Portfolio Management & Construction":
            for sub_skill, sub_data in skill_data.get("sub_skills", {}).items():
                if sub_skill.lower() in lower_context or any(a.lower() in lower_context for a in sub_data.get("aliases", [])):
                    enriched_data["details"]["Focus"] = sub_skill
                    break
            if "python" in lower_context: enriched_data["details"]["Tool"] = "Python"
            if "r" in lower_context: enriched_data["details"]["Tool"] = "R" # Could be a list of tools

        if skill_id == "Communication & Presentation":
            if "client" in lower_context and ("pitch" in lower_context or "recommendation" in lower_context): enriched_data["details"]["Audience"] = "External Clients"
            elif "committee" in lower_context or "management" in lower_context: enriched_data["details"]["Audience"] = "Internal Stakeholders"
            if "persuaded" in lower_context or "influenced" in lower_context or "resulted in" in lower_context: enriched_data["details"]["Impact"] = "Decision-Making"
        
        if skill_id == "Market & Sector Expertise":
            for sector_example in skill_data.get("examples", []):
                # Simple check for direct sector mention in context
                if sector_example.lower().split(' ')[0] in lower_context: # e.g., "tmt" in "tmt sector"
                    enriched_data["details"]["Sector_Expertise"] = sector_example
                    break
            
        if skill_id == "Due Diligence":
            for component in skill_data.get("key_components", []):
                if component.lower() in lower_context:
                    if "Components_Mentioned" not in enriched_data["details"]:
                        enriched_data["details"]["Components_Mentioned"] = []
                    enriched_data["details"]["Components_Mentioned"].append(component)

        # Add generic context keywords that are highly relevant
        for keyword in ["implementation", "strategy", "decision", "recommendation", "optimization", "automation", 
                        "reporting", "compliance", "analysis", "development", "research", "valuation", "execution"]:
            if keyword in lower_context:
                if keyword not in enriched_data["context_keywords"]: # Avoid duplicates
                    enriched_data["context_keywords"].append(keyword)

        return enriched_data

    def find_skill(self, phrase: str, full_sentence_context: str = "") -> List[Dict[str, Any]]:
        """
        Intelligently identifies skills, their levels, and context from a phrase and its surrounding sentence.
        Returns a list of detected skills, as a single phrase might imply multiple skills.
        """
        detected_skills = []
        lower_phrase = phrase.lower().strip()
        lower_context = full_sentence_context.lower().strip()

        # Iterate through the entire graph to find matches
        for skill_id, skill_data in self.graph.items():
            found_in_this_skill_tree = False

            # 1. Check for exact match with main skill ID or its aliases
            if lower_phrase == skill_id.lower() or any(alias.lower() == lower_phrase for alias in skill_data.get("aliases", [])):
                detected_skills.append(self._enrich_skill_data(skill_id, skill_data, phrase, full_sentence_context))
                found_in_this_skill_tree = True
            
            # 2. Check for matches within sub_skills (e.g., "Fundamental Analysis" inside "Investment Research")
            if "sub_skills" in skill_data and not found_in_this_skill_tree:
                for sub_skill_name, sub_skill_data in skill_data["sub_skills"].items():
                    if lower_phrase == sub_skill_name.lower() or any(alias.lower() == lower_phrase for alias in sub_skill_data.get("aliases", [])):
                        detected_skills.append(self._enrich_skill_data(skill_id, skill_data, phrase, full_sentence_context)) # Attach to parent skill
                        found_in_this_skill_tree = True
                        break

            # 3. Check for matches within specific_implementations/models/tools/components (e.g., "DCF" inside "Financial Modeling")
            if "models" in skill_data and not found_in_this_skill_tree:
                for model_name, model_data in skill_data["models"].items():
                    if lower_phrase == model_name.lower() or any(alias.lower() == lower_phrase for alias in model_data.get("aliases", [])):
                        detected_skills.append(self._enrich_skill_data(skill_id, skill_data, phrase, full_sentence_context))
                        found_in_this_skill_tree = True
                        break
            if "key_components" in skill_data and not found_in_this_skill_tree: # For Due Diligence components etc.
                 if lower_phrase in [comp.lower() for comp in skill_data["key_components"]]:
                    detected_skills.append(self._enrich_skill_data(skill_id, skill_data, phrase, full_sentence_context))
                    found_in_this_skill_tree = True

            # 4. Keyword-based detection using full context (where LLM is crucial in real system)
            # This is a broader context check. Avoid re-adding if a direct match already added this skill_id.
            if not found_in_this_skill_tree:
                # Combine skill_id, aliases, and examples for broader keyword search within the context
                keywords_to_check_in_context = [skill_id.lower()] + [a.lower() for a in skill_data.get("aliases", [])]
                if "examples" in skill_data: # For Market & Sector Expertise
                    keywords_to_check_in_context.extend([e.lower() for e in skill_data["examples"]])
                
                # Check for the primary phrase or its root in the context
                if lower_phrase in lower_context:
                    # Check if any keyword from this skill_id is present in the context, not just the phrase
                    for kw in keywords_to_check_in_context:
                        if kw in lower_context:
                            detected_skills.append(self._enrich_skill_data(skill_id, skill_data, phrase, full_sentence_context))
                            found_in_this_skill_tree = True
                            break # Found it, move to next skill_id

        # Remove duplicates based on skill name. If multiple phrases map to same skill, keep first or most detailed.
        unique_skills = []
        seen_names = set()
        for skill in detected_skills:
            if skill["name"] not in seen_names:
                unique_skills.append(skill)
                seen_names.add(skill["name"])

        # If nothing specific was detected, return a generic unknown skill
        if not unique_skills:
            return [{"name": phrase, "type": "Unknown", "detected_phrase": phrase, "level": "unknown", "applications": [], "related_skills": [], "context_keywords": [], "details": {}}]
        
        return unique_skills

## **3. Custom Tool for Buyside Skill Graph**

In [ ]:
class BuysideSkillTool(BaseTool):
    name: str = "Buyside Skill Analyzer"
    description: str = (
        "Analyzes a given skill phrase from a buyside candidate's resume within its full sentence context "
        "to identify standardized skill, level, applications, and relevant details using a sophisticated "
        "Buyside Asset Management Skill Graph. Returns a dictionary of detected skill attributes."
    )
    skill_graph: BuysideAssetManagementSkillGraph = Field(default_factory=BuysideAssetManagementSkillGraph)

    def _run(self, phrase: str, context_sentence: str) -> Dict[str, Any]:
        """
        Runs the tool with the given phrase and context.
        This method is called by the Agent when it uses the tool.
        """
        # print(f"Tool Used: Buyside Skill Analyzer for phrase='{phrase}', context='{context_sentence}'") # For internal tool debugging
        results = self.skill_graph.find_skill(phrase, context_sentence)
        
        # For simplicity, we return the first detected skill if multiple are found,
        # or a generic unknown if none.
        if results:
            return results[0] 
        return {"name": phrase, "type": "Unknown", "level": "unknown", "details": {}, "context_keywords": []}

# Instantiate the specialized tool for use by agents
buyside_skill_analyzer_tool = BuysideSkillTool()

## **4. Pydantic Models for Structured Outputs** These define the expected format for the AI's reports.

In [ ]:
class SkillReport(BaseModel):
    required_skills: List[str] = Field(description="List of skills explicitly required by the job description.")
    possessed_skills: List[Dict[str, Any]] = Field(description="List of skills the candidate possesses, with details (name, level, evidence, related_context, details like tools/sectors/models from the skill graph).")
    missing_skills: List[str] = Field(description="List of required skills the candidate appears to be missing.")
    overall_skill_match_rating: int = Field(..., ge=1, le=10, description="Overall numerical rating (1-10) of the candidate's skill match against the job description.")

class CompanyMatchReport(BaseModel):
    alignment_points: List[str] = Field(description="List of specific points where the candidate's profile aligns with the company's culture, values, or investment philosophy.")
    misalignment_points: List[str] = Field(description="List of specific points where there might be a mismatch or areas for concern.")
    overall_company_fit_rating: int = Field(..., ge=1, le=10, description="Overall numerical rating (1-10) of the candidate's fit with the company culture and strategic needs.")

class HiringManagerPreferencesReport(BaseModel):
    alignment_points: List[str] = Field(description="Specific aspects of the candidate's profile that align with the hiring manager's preferences, including how they enhance the existing team.")
    areas_for_further_probe: List[str] = Field(description="Questions or areas that need further clarification during an interview, based on preferences and team fit.")
    overall_preference_alignment_rating: int = Field(..., ge=1, le=10, description="Overall numerical rating (1-10) of the candidate's alignment with the hiring manager's specific preferences and team needs.")


## **5. Input Data (Candidate CV, Job & Company Details)** This provides all the necessary context for the AI agents.

In [ ]:
OUTPUT_DIRECTORY = "output_reports" # Directory to save final reports

candidate_resume_text = """
Anna Chen
[anna.chen@email.com] | [+886-9xx-xxx-xxx] | [LinkedIn.com/in/annacheninvest]

Summary
Highly analytical and results-driven Investment Analyst with 5 years of experience in equity research and financial modeling, specializing in the TMT and Consumer sectors. Proven ability to conduct deep fundamental analysis, construct robust valuation models (DCF, LBO, Comps), and present actionable investment recommendations to senior management and external clients. Proficient in Python for quantitative analysis and portfolio risk management. Seeking to leverage strong analytical capabilities and market insights to contribute to a leading asset management firm.

---

Experience

Senior Investment Analyst | Global Equities Fund, Taipei, Taiwan
Jan 2022 – Present
* Led equity research for a portfolio of TMT sector stocks, covering over 20 companies with a combined market cap of $50B+.
* Built and maintained complex DCF and LBO models to support investment decisions for private and public equity deals.
* Conducted due diligence on potential investment targets, analyzing financial statements, industry trends, and competitive landscapes.
* Presented comprehensive investment recommendations to the Investment Committee, resulting in successful deployment of over $150M capital.
* Utilized Bloomberg Terminal and Refinitiv Eikon for real-time market data and news.
* Developed Python scripts for automating data extraction from financial statements and backtesting basic quantitative strategies.

Investment Analyst | Regional Investment Bank, Hong Kong
Aug 2019 – Dec 2021
* Assisted senior analysts with fundamental analysis across various sectors, including healthcare and industrials.
* Constructed comparable company analysis (Comps) and precedent transaction models for M&A advisory.
* Prepared client pitches and offering memoranda for IPOs and private placements.
* Gained experience in portfolio optimization principles and basic risk metrics.

---

Education

Master of Finance | National Taiwan University, Taipei, Taiwan
Sep 2018 – Jun 2019
* Specialization in Quantitative Finance.

Bachelor of Business Administration (Finance) | National Chengchi University, Taipei, Taiwan
Sep 2014 – Jun 2018

---

Skills
* Investment Research: Fundamental Analysis, Quantitative Analysis, Due Diligence, Sector Analysis (TMT, Consumer)
* Financial Modeling: DCF, LBO, Comparable Company Analysis, Scenario Analysis, Sensitivity Analysis, Forecasting
* Data & Tools: Python (Pandas, NumPy, Matplotlib), SQL, Microsoft Excel (Advanced), Bloomberg Terminal, Refinitiv Eikon, Power BI
* Portfolio Management: Risk Management, Asset Allocation (basic), Performance Attribution
* Soft Skills: Communication, Presentation, Critical Thinking, Problem-Solving, Stakeholder Management
"""
candidate_name = "Anna Chen" # Directly set for demo

job_position = "Investment Analyst (Buyside)"
job_description = """
We are seeking a highly driven and analytical Investment Analyst to join our buyside team, focusing on global equities.
The ideal candidate will possess strong fundamental and quantitative research skills,
demonstrated experience in building sophisticated financial models (DCF, LBO, Comps),
and a proven ability to present clear, data-driven investment recommendations.
Proficiency in Python for quantitative analysis and portfolio risk management is highly valued.
Experience in TMT or Consumer sectors is a plus.
"""

company_description = """
Our company is a leading independent asset management firm with a strong focus on fundamental research and long-term value creation.
We manage a diverse portfolio of global equities for institutional and high-net-worth clients.
Our investment philosophy emphasizes deep dives into companies, rigorous valuation, and a strong understanding of macro trends.
We foster a collaborative, intellectually curious environment where analytical rigor and independent thought are highly prized.
"""

# UPDATED: Hiring Manager Preferences including team structure and skillsets
hiring_manager_preferences = """
The Head of Equities is looking for an Investment Analyst who is not only technically sound in modeling and research
but also possesses strong independent thinking and a proactive approach to idea generation.
Excellent presentation and communication skills are paramount for conveying complex investment theses to both internal committees and external clients.
A candidate who can articulate their investment philosophy, challenge assumptions, and contribute to a rigorous debate is highly preferred.
Experience with both qualitative fundamental analysis and quantitative tools (e.g., Python for risk analysis or backtesting) is a significant advantage.
Cultural fit within a lean, high-performing team is crucial; we value intellectual honesty, humility, and a strong work ethic.

--- Existing Team Context for Investment Analyst Role ---
Our current Equities Research team comprises 7 analysts, reporting directly to the Head of Equities.
**Team Structure:** Flat hierarchy within the research team. Analysts often lead projects and collaborate cross-functionally.
**Current Key Skillsets within the Team:**
- **Analyst A (Senior):** Deep fundamental research (Consumer, Industrials), advanced financial modeling (Excel), M&A advisory.
- **Analyst B (Senior):** Portfolio construction, macro overlay strategies, client relationship management.
- **Analyst C (Mid-level):** Strong quantitative analysis (R, basic Python), risk attribution, market data interpretation.
- **Analyst D (Mid-level):** Sector expert (Healthcare), regulatory analysis, due diligence.
- **Analyst E (Junior):** Basic financial modeling, data collection, report generation.
- **Analyst F (Junior):** Supports trading desk with quick research, market microstructure.
- **Analyst G (Junior):** Specialized in ESG research and integration into investment processes.

**Identified Team Skill Gaps / Areas for Enhancement with New Hire:**
1.  **Advanced Python for Systematic Strategies:** We need stronger capabilities in developing and backtesting systematic or factor-based strategies using Python beyond basic data analysis.
2.  **TMT/Emerging Tech Sector Expertise:** Deeper fundamental and trend analysis in cutting-edge technology and emerging markets within the TMT space.
3.  **High-Impact Client-Facing Presentation:** While we have good communicators, we seek someone who can truly captivate and persuade sophisticated institutional clients with complex investment narratives.
4.  **Proactive Idea Generation & Debate:** A candidate who can consistently bring fresh, contrarian ideas and engage in rigorous, constructive debate, challenging existing assumptions.
"""

company_knowledge_base_content = """
**Company History & Vision:**
Founded in [Year], our firm has grown to manage over [X] Billion AUM, recognized for our contrarian views and disciplined investment process. Our vision is to deliver superior, risk-adjusted returns by identifying mispriced assets globally. We are expanding our research capabilities in emerging markets.

**Industry Knowledge (Asset Management):**
The asset management industry is highly competitive, characterized by rapid information flow, technological disruption (e.g., AI in finance, big data analytics), and increasing client demand for transparency and bespoke solutions. Active management requires deep insight and differentiated research.

**Philosophy of Talent Acquisition:**
We seek individuals who are intellectually curious, possess a strong intrinsic motivation for financial markets, and demonstrate impeccable integrity. Our hiring process is rigorous, emphasizing analytical prowess, independent thought, and the ability to articulate complex ideas clearly. We believe in nurturing talent through mentorship and exposure to diverse investment strategies.

**Core Functions of Investment Team (Relevant to Investment Analyst):**
- **Research:** Deep fundamental and quantitative analysis to identify investment opportunities.
- **Portfolio Management:** Constructing and optimizing portfolios based on research insights and risk parameters.
- **Trading:** Executing investment decisions efficiently.
- **Client Relations:** Communicating investment strategies and performance to clients.

**Existing Investment Team Context (General for Firm):**
The current equities research team consists of 7 analysts covering various sectors, reporting to the Head of Equities.
Team strengths: Strong fundamental research in traditional sectors (energy, industrials).
Areas for Growth/New Hire Impact: We are looking for someone who can:
1.  Bring stronger quantitative research capabilities, particularly in Python for systematic strategy development or advanced risk modeling.
2.  Enhance our TMT/Consumer sector coverage with deep insights.
3.  Elevate our client-facing presentation skills and ability to articulate complex theses persuasively.
4.  Contribute fresh, independent ideas and challenge existing assumptions rigorously.
The ideal candidate will be a proactive, analytical, and articulate individual who can immediately contribute to alpha generation and strategic discussions.
"""

## **6. Agent Definitions** Each AI agent has a specific role, goal, and backstory to guide its reasoning.

In [ ]:
skill_screening_agent = Agent(
    role="Buyside Skill Screening Agent",
    goal=f"Screen the candidate's skills against the '{job_position}' job description, using a specialized Buyside skill graph for deep analysis.",
    backstory=(
        "You are a meticulous skill screening agent specializing in Buyside Asset Management roles. "
        "Your expertise lies in dissecting job descriptions and candidate resumes to identify precise skill matches and gaps, "
        "going beyond keywords to understand depth, application, and context. "
        "You utilize a sophisticated Buyside skill graph to standardize, categorize, and assess skills, "
        "ensuring accurate and consistent assessments for highly specialized financial roles. "
        "You provide clear, evidence-based assessments of a candidate's analytical, technical, and strategic capabilities."
    ),
    llm=llm_config,
    verbose=True, # Show internal thoughts and tool usage
    max_iterations=8, # Increased iterations for complex analysis
    max_retries=3,
    temperature=0.5,
    tools=[buyside_skill_analyzer_tool] # The skill agent specifically uses your custom tool
)

company_matching_agent = Agent(
    role="Buyside Company Matching Agent",
    goal=f"Evaluate how well the candidate aligns with a leading asset management firm's needs and culture for the '{job_position}' role, leveraging deep company knowledge.",
    backstory=(
        "You are a strategic company matching agent with an in-depth understanding of our asset management firm's investment philosophy, "
        "history, vision, industry position, and operational philosophy. You analyze a candidate's career trajectory, "
        "the types of financial institutions they've worked for, and their approach to investment, "
        "assessing their fit with our firm's investment values, analytical rigor, and collaborative culture. "
        "You identify cultural and strategic alignment, always considering our unique buyside business context. "
        f"\n\n--- COMPANY KNOWLEDGE BASE ---\n{company_knowledge_base_content}"
    ),
    llm=llm_config,
    verbose=True, # Show internal thoughts
    max_iterations=5,
    max_retries=3,
    temperature=0.7,
)

hiring_manager_preferences_agent = Agent(
    role="Buyside Hiring Manager Preferences Agent",
    goal=f"Assess the candidate's profile against the specific preferences of the Head of Equities for the '{job_position}' role, **critically evaluating how they fit into and enhance the existing team structure and skillsets.**",
    backstory=(
        "You are a specialized agent focused on the subjective yet critical aspects of hiring in asset management, "
        "with a keen eye for intellectual curiosity, independent thought, and communication prowess. "
        "You meticulously compare a candidate's analytical approach, presentation skills, and problem-solving abilities "
        "against the hiring manager's stated preferences, and crucially, **how they would integrate into and enhance the existing, specific research team outlined by the hiring manager, filling identified skill gaps and fostering desired team dynamics.** "
        "You provide a qualitative fit analysis, emphasizing intellectual honesty and contribution to debate."
    ),
    llm=llm_config,
    verbose=True, # Show internal thoughts
    max_iterations=5,
    max_retries=3,
    temperature=0.7,
)

hiring_decision_maker_agent = Agent(
    role="Lead Hiring Decision Maker (Buyside)",
    goal=f"Consolidate all reports and make a final hiring recommendation for the '{job_position}' role based on a weighted assessment, justifying the decision rigorously.",
    backstory=(
        "You are the Lead Hiring Decision Maker for a Buyside Asset Management firm. Your ultimate responsibility is to ensure "
        "only the best talent joins our high-performing investment team. You synthesize complex information from various "
        "specialized agents to form a holistic view of the candidate. Your decisions are always data-driven, strategic, "
        "and aligned with the firm's long-term vision for talent acquisition in asset management."
        f"\n\n--- FIRM'S OVERALL HIRING STRATEGY ---\n{company_knowledge_base_content}" # Reuse for broader context
    ),
    llm=llm_config,
    verbose=True, # Show internal thoughts
    max_iterations=3,
    max_retries=2,
    temperature=0.3, # Lower temperature for more deterministic decision-making
)

print("All Agents Defined (with specialized Buyside Knowledge Base and Tool integration).")

## **7. Task Definitions**
These define the specific steps and outputs for each agent.

In [ ]:
skill_screening_task = Task(
    description=f"""
    Analyze the provided candidate resume for '{candidate_name}' and deeply screen their skills against the following job description for the '{job_position}' role.
    For every skill or relevant experience you identify in the resume, **you MUST use the 'Buyside Skill Analyzer' tool** by calling it with the exact phrase from the resume and its full sentence context. Example tool call: `self.tools[0].run(phrase='equity research', context_sentence='Led equity research for a portfolio of TMT sector stocks.')`

    --- JOB DESCRIPTION ---
    {job_description}
    --- CANDIDATE RESUME ({candidate_name}) ---
    {candidate_resume_text}

    Identify the core skills (e.g., Investment Research, Financial Modeling, Portfolio Management, Communication, Due Diligence, Market Expertise) explicitly mentioned or strongly implied as required in the job description.
    Then, extract all relevant skills from the candidate's resume, providing brief evidence or context from the resume for each.
    **Crucially, for each possessed skill, detail its type, detected level, specific applications, and any tools/sectors/models identified by the 'Buyside Skill Analyzer' tool.**
    Clearly state which of the job description's required skills the candidate possesses and which are missing.
    Finally, provide a numerical rating from 1 to 10 on the overall skill match, emphasizing depth and applicability to a buyside environment.
    """,
    agent=skill_screening_agent,
    output_pydantic=SkillReport, # Use Pydantic model for output
    verbose=True
)

company_matching_task = Task(
    description=f"""
    Evaluate the candidate '{candidate_name}'s background against the company's needs and culture based on the provided descriptions for the '{job_position}' role.
    Leverage your deep understanding of the company's history, vision, industry, and philosophy (from your backstory), especially focusing on our active management philosophy, intellectual curiosity, and rigorous analysis.
    --- COMPANY DESCRIPTION ---
    {company_description}
    --- CANDIDATE RESUME ({candidate_name}) ---
    {candidate_resume_text}

    Assess the candidate's past company experience, roles, and demonstrated approach to investments against the hiring company's focus areas (e.g., global equities, fundamental research, long-term value creation), values (intellectual curiosity, analytical rigor), and operational regions.
    Provide a rating from 1 to 10 on overall company fit, along with detailed reasoning, explicitly referencing how the candidate's profile aligns with the company's knowledge base and investment philosophy.
    """,
    agent=company_matching_agent,
    output_pydantic=CompanyMatchReport,
    verbose=True
)

hiring_manager_preferences_task = Task(
    description=f"""
    Evaluate the candidate '{candidate_name}'s resume against the Head of Equities' specific preferences for the '{job_position}' role.
    --- HIRING MANAGER PREFERENCES & TEAM CONTEXT ---
    {hiring_manager_preferences}
    --- CANDIDATE RESUME ({candidate_name}) ---
    {candidate_resume_text}

    Assess how well the candidate's profile aligns with preferences such as independent thinking, proactive idea generation, presentation/communication skills for complex investment theses, and cultural fit (intellectual honesty, humility, work ethic) within a lean, high-performing team.
    **Crucially, evaluate how the candidate's experience and analytical approach would fit into and enhance the existing team structure and dynamics, explicitly addressing the identified team skill gaps (e.g., stronger quantitative research, TMT/Consumer coverage, client-facing presentation) and whether the candidate can fill these or bring complementary strengths.**
    Provide specific examples or inferences from the resume where possible for each preference and team fit aspect.
    Provide a rating from 1 to 10 on overall hiring manager preference alignment, significantly weighting the intellectual fit, communication impact, and cultural integration.
    """,
    agent=hiring_manager_preferences_agent,
    output_pydantic=HiringManagerPreferencesReport,
    verbose=True
)

final_hiring_decision_task = Task(
    description=f"""
    You are the Lead Hiring Decision Maker for a Buyside Asset Management firm. Your task is to consolidate the detailed evaluations provided by the Skill Screening Agent,
    the Company Matching Agent, and the Hiring Manager Preferences Agent for candidate {candidate_name}, for the '{job_position}' role.
    Leverage your deep understanding of the company's overall context and hiring philosophy (from your backstory), focusing on attracting top-tier investment talent.

    **Input Reports (Pydantic Objects):**
    - Skill Screening Report: This will be a Pydantic object (SkillReport) with fields like `overall_skill_match_rating`.
    - Company Matching Report: This will be a Pydantic object (CompanyMatchReport) with fields like `overall_company_fit_rating`.
    - Hiring Manager Preferences Report: This will be a Pydantic object (HiringManagerPreferencesReport) with fields like `overall_preference_alignment_rating`.

    **Step 1: Determine Dynamic Weights**
    For an 'Investment Analyst (Buyside)' role, **analytical rigor, strategic thinking, and cultural/team fit are paramount**.
    Determine the optimal weighting for the following three criteria. The sum of the weights must be 10 points.
    - Skill Matching: [X]/10 (emphasis on deep analytical skills, modeling, quant tools)
    - Company Matching: [Y]/10 (emphasis on alignment with investment philosophy, research culture)
    - Hiring Manager Preference: [Z]/10 (emphasis on independent thought, communication impact, and team integration/contribution to debate, specifically addressing team skill gaps)
    Provide your clear reasoning for the chosen weights.

    **Step 2: Extract Individual Scores & Calculate Overall Weighted Score**
    Access the numerical ratings directly from the provided Pydantic objects.
    Calculate the overall weighted score for the candidate using your determined weights and the extracted scores.
    Show your calculation clearly.

    **Step 3: Formulate Final Recommendation and Justification**
    Based on the overall weighted score and your holistic judgment, formulate a clear and concise final hiring recommendation
    (e.g., "Highly Recommended for Interview", "Recommended for Interview", "Consider for Further Review", "Not Recommended").
    Provide a comprehensive justification for your recommendation, referencing the individual scores, the overall weighted score,
    and any critical insights from the specialized reports, explicitly linking back to the firm's strategic needs, investment philosophy, and the specific needs of the Head of Equities.

    Your Final Output MUST be a structured report in Markdown format including:
    # Final Hiring Recommendation Report for {candidate_name}
    ## Job Position: {job_position}
    ## Candidate Name: {candidate_name}

    ### 1. Determined Weights for Evaluation Criteria:
    - Skill Matching: [X]/10
    - Company Matching: [Y]/10
    - Hiring Manager Preference: [Z]/10
    *(Sum X+Y+Z must be 10. Explain reasoning for these weights.)*

    ### 2. Individual Agent Scores:
    - Skill Match Score: [Score from Skill Screening Agent]/10
    - Company Fit Score: [Score from Company Matching Agent]/10
    - Hiring Manager Preference Alignment Score: [Score from Hiring Manager Preferences Agent]/10

    ### 3. Overall Weighted Score Calculation:
    [Show your calculation here, e.g., (Skill Score * Skill Weight) + (Company Score * Company Weight) + (Preference Score * Preference Weight) / 10]
    **Overall Weighted Score:** [Calculated Score]

    ### 4. Final Recommendation:
    **[Highly Recommended for Interview / Recommended for Interview / Consider for Further Review / Not Recommended]**

    ### 5. Justification for Recommendation:
    [Detailed explanation based on scores, overall fit, and critical insights from all reports, referencing company knowledge and hiring manager priorities.]
    """,
    expected_output=f"""A detailed final hiring recommendation report for {candidate_name} for the {job_position} role,
                            including dynamically determined weights, individual agent scores, overall weighted score,
                            and a justified final recommendation, formatted in Markdown.""",
    agent=hiring_decision_maker_agent,
    context=[skill_screening_task, company_matching_task, hiring_manager_preferences_task],
    verbose=True
)

print("All Tasks Defined for Anna Chen.")

## **8. Assemble the Crew and Kickoff**
This section brings everything together and runs the automated screening process.

In [ ]:
if __name__ == "__main__":
    # Ensure output directory exists
    os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)

    print(f"\n\n{'='*80}")
    print(f"Processing Resume for: {candidate_name}")
    print(f"{'='*80}\n")

    recruitment_crew = Crew(
        agents=[
            skill_screening_agent,
            company_matching_agent,
            hiring_manager_preferences_agent,
            hiring_decision_maker_agent
        ],
        tasks=[
            skill_screening_task,
            company_matching_task,
            hiring_manager_preferences_task,
            final_hiring_decision_task
        ],
        process=Process.sequential, # Agents execute tasks sequentially
        verbose=True, # Crew-level verbose setting (shows overall process)
        full_output=True # Get full details including agent trajectory
    )

    print(f"\n\n################################################")
    print(f"## Starting the CV Screener Crew Process for {candidate_name}...")
    print("################################################\n")

    # Kickoff the crew process
    crew_result = recruitment_crew.kickoff()

    print(f"\n\n################################################")
    print(f"## Crew Process Finished for {candidate_name}")
    print(f"################################################\n")
    
    # crew_result directly contains the final string output from the last task
    final_report_content = crew_result 

    print(final_report_content)

    # Save the final report to a file
    output_file_name = f"{candidate_name.replace(' ', '_')}_Buyside_Hiring_Report.md"
    output_file_path = os.path.join(OUTPUT_DIRECTORY, output_file_name)
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write(final_report_content)
    print(f"\nFinal report saved to: {output_file_path}")

    print("\n\nDemo finished.")